In [33]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import pickle
import time

# prevent tensorflow from using GPU. Otherwise, run out of memory
# https://stackoverflow.com/questions/44552585/prevent-tensorflow-from-accessing-the-gpu
# os.environ["CUDA_VISIBLE_DEVICES"]="-1"

import tensorflow_hub as hub
import tensorflow as tf

import keras
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional,Input, Embedding, Flatten
from keras import Model
from keras.optimizers import RMSprop
from tensorflow.keras.callbacks import TensorBoard
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.utils import to_categorical

elmo = hub.Module("https://tfhub.dev/google/elmo/2")

In [41]:
# Load Pickles
pickle_path = '/home/tim/Documents/Sentiment/Data/pickles'

#x_train
pickle_in = open(os.path.join(pickle_path,'x_train.pickle'),"rb")
x_train = pickle.load(pickle_in)

#y_train
pickle_in = open(os.path.join(pickle_path,'y_train.pickle'),"rb")
y_train = pickle.load(pickle_in)

#x_val
pickle_in = open(os.path.join(pickle_path,'x_dev.pickle'),"rb")
x_val = pickle.load(pickle_in)

#y_val
pickle_in = open(os.path.join(pickle_path,'y_dev.pickle'),"rb")
y_val = pickle.load(pickle_in)

#x_test
pickle_in = open(os.path.join(pickle_path,'x_test.pickle'),"rb")
x_test = pickle.load(pickle_in)

#y_test
pickle_in = open(os.path.join(pickle_path,'y_test.pickle'),"rb")
y_test = pickle.load(pickle_in)

In [42]:
# convert y values to one-hot encoding
y_train = to_categorical(y_train - y_train.min())
y_val = to_categorical(y_val - y_val.min())
y_test = to_categorical(y_test - y_test.min())

In [43]:
# build model

lstm_1_units = 100
lstm_2_units = 50
dense_1_units = 100
dense_2_units = 7

model = Sequential()
model.add(Bidirectional(LSTM(lstm_1_units, activation='relu', return_sequences=True), input_shape=(80,512)))
model.add(Bidirectional(LSTM(lstm_2_units,activation='relu', return_sequences=False)))
model.add(Dense(dense_1_units, activation = 'relu'))
model.add(Dense(dense_2_units, activation = 'softmax'))
model.compile(optimizer='adam',loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_23 (Bidirectio (None, 80, 200)           490400    
_________________________________________________________________
bidirectional_24 (Bidirectio (None, 100)               100400    
_________________________________________________________________
dense_23 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_24 (Dense)             (None, 7)                 707       
Total params: 601,607
Trainable params: 601,607
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# TRAIN
# model name
NAME = "biLSTM_L1={}_L2={}_d1={}_d2={}_{}".format(lstm_1_units,
                                                    lstm_2_units,
                                                    dense_1_units,
                                                    dense_2_units,
                                                    int(time.time()))
print(NAME)

# # callbacks
tensorboard = TensorBoard(log_dir="logs/biLSTM1/{}".format(NAME))

b = 64 # batch size

model.fit(x_train, y_train,
          epochs = 100,
          validation_data=(x_val,y_val),
          shuffle=True,
          batch_size=b,
          verbose = 1,
          callbacks=[tensorboard])

# keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=0, verbose=0, mode='auto', baseline=None, restore_best_weights=False)


# loss, accuracy = model.evaluate(X_train, Y_train, verbose=2)
# print('Accuracy: %f' % (accuracy*100))

biLSTM_L1=100_L2=50_d1=100_d2=7_1555095463
Train on 1181 samples, validate on 449 samples
Epoch 1/100
1181/1181 [==============================] - 6s 5ms/step - loss: 14.1938 - acc: 0.1194 - val_loss: 14.1437 - val_acc: 0.1225
Epoch 2/100
1181/1181 [==============================] - 5s 5ms/step - loss: 14.0709 - acc: 0.1270 - val_loss: 14.2514 - val_acc: 0.1158
Epoch 3/100
1181/1181 [==============================] - 6s 5ms/step - loss: 14.0027 - acc: 0.1312 - val_loss: 14.2514 - val_acc: 0.1158
Epoch 4/100
1181/1181 [==============================] - 6s 5ms/step - loss: 13.9754 - acc: 0.1329 - val_loss: 14.2514 - val_acc: 0.1158
Epoch 5/100
1181/1181 [==============================] - 6s 5ms/step - loss: 13.9754 - acc: 0.1329 - val_loss: 14.2514 - val_acc: 0.1158
Epoch 6/100
1181/1181 [==============================] - 5s 5ms/step - loss: 13.9754 - acc: 0.1329 - val_loss: 14.2514 - val_acc: 0.1158
Epoch 7/100
1181/1181 [==============================] - 5s 5ms/step - loss: 13.9754 - a